STEP 2:

Compute 128-d face embeddings to quantify a face



In [28]:
# import the necessary packages
from imutils import paths
import numpy as np
import imutils
import pickle
import cv2
import os

In [29]:
# load our serialized face detector from disk
print("[INFO] loading face detector...")
protoPath = "model/deploy.prototxt"
modelPath = "model/res10_300x300_ssd_iter_140000.caffemodel"
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

# load our serialized face embedding model from disk
print("[INFO] loading face recognizer...")
# Path to the OpenCV deep learning Torch embedding model. 
# This model will allow us to extract a 128-D facial embedding vector.
embedder = cv2.dnn.readNetFromTorch("model/openface_nn4.small2.v1.t7")

[INFO] loading face detector...
[INFO] loading face recognizer...


In [30]:
# grab the paths to the input images in our dataset
print("[INFO] quantifying faces...")
imagePaths = list(paths.list_images("captured"))

# initialize our lists of extracted facial embeddings and
# corresponding people names
knownEmbeddings = []
knownNames = []

# initialize the total number of faces processed
total = 0

[INFO] quantifying faces...


In [31]:
# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
	# extract the person name from the image path
	print("[INFO] processing image {}/{}".format(i + 1, len(imagePaths)))
	name = imagePath.split(os.path.sep)[-2]
    
	# load the image, resize it to have a width of 600 pixels (while
	# maintaining the aspect ratio), and then grab the image
	# dimensions
	image = cv2.imread(imagePath)
	image = imutils.resize(image, width=600)
	(h, w) = image.shape[:2]

	# construct a blob from the image
	imageBlob = cv2.dnn.blobFromImage(
		cv2.resize(image, (300, 300)), 1.0, (300, 300),
		(104.0, 177.0, 123.0), swapRB=False, crop=False)
    
	# apply OpenCV's deep learning-based face detector to localize
	# faces in the input image
	detector.setInput(imageBlob)
	detections = detector.forward()

	# ensure at least one face was found
	if len(detections) > 0:
		# we're making the assumption that each image has only ONE
		# face, so find the bounding box with the largest probability
		i = np.argmax(detections[0, 0, :, 2])
		confidence = detections[0, 0, i, 2]
        
		# ensure that the detection with the largest probability also
		# means our minimum probability test (thus helping filter out
		# weak detections)
		if confidence > 0.5:
			# compute the (x, y)-coordinates of the bounding box for
			# the face
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")
            
			# extract the face ROI and grab the ROI dimensions
			face = image[startY:endY, startX:endX]
			(fH, fW) = face.shape[:2]
            
			# ensure the face width and height are sufficiently large
			if fW < 20 or fH < 20:
				continue

			# construct a blob for the face ROI, then pass the blob
			# through our face embedding model to obtain the 128-d
			# quantification of the face
			faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,
				(96, 96), (0, 0, 0), swapRB=True, crop=False)
			embedder.setInput(faceBlob)
			vec = embedder.forward()
            
			# add the name of the person + corresponding face
			# embedding to their respective lists
			knownNames.append(name)
			knownEmbeddings.append(vec.flatten())
			total += 1

[INFO] processing image 1/138
[INFO] processing image 2/138
[INFO] processing image 3/138
[INFO] processing image 4/138
[INFO] processing image 5/138
[INFO] processing image 6/138
[INFO] processing image 7/138
[INFO] processing image 8/138
[INFO] processing image 9/138
[INFO] processing image 10/138
[INFO] processing image 11/138
[INFO] processing image 12/138
[INFO] processing image 13/138
[INFO] processing image 14/138
[INFO] processing image 15/138
[INFO] processing image 16/138
[INFO] processing image 17/138
[INFO] processing image 18/138
[INFO] processing image 19/138
[INFO] processing image 20/138
[INFO] processing image 21/138
[INFO] processing image 22/138
[INFO] processing image 23/138
[INFO] processing image 24/138
[INFO] processing image 25/138
[INFO] processing image 26/138
[INFO] processing image 27/138
[INFO] processing image 28/138
[INFO] processing image 29/138
[INFO] processing image 30/138
[INFO] processing image 31/138
[INFO] processing image 32/138
[INFO] processing

In [32]:
# dump the facial embeddings + names to disk
print("[INFO] serializing {} encodings...".format(total))
data = {"embeddings": knownEmbeddings, "names": knownNames}
f = open("output/embed.pk", "wb")
f.write(pickle.dumps(data))
f.close()

[INFO] serializing 135 encodings...
